# RNN(Recurrent Neural Networks)

- 앞의 입력 값을 기억
- 앞에서 나온 입력에 대한 결과가 뒤에서 나오는 입력 값에 영향을 줌
- 활성함수로 하이퍼볼릭탄젠트 사용, 미분하면 전체값이 0 ~ 0.5로 나와서 경사소실문제 야기하지만,   
마지막(가까이 있는) 입력일수록 더 많이 영향을 끼치게하는 효과도 있어서 사용    
- 문제점 : 예측하려는 것과 가장 높은 상관 관계를 가진 입력 값이 앞쪽에 있으면 미분으로 인한 경사소실로 예측하기 어려움

- 해결방안 : LSTM(Long Short Term Memory)
- LSTM : 기억을 관리 해줌 (데이터가 입력되는 순서에 따라서만 중요도를 생각하는 RNN과 달리   
기존의 들어왔던 입력들의 중요도를 중간에 따로 계산함) 그럼 입력 순서에 상관없이 중요도가 설정됨

# [예제] 단어 예측
- 5글자로 된 단어 중 앞의 4글자를 입력하면 마지막 글자를 출력하는 모델
- 글자 great,  freak, right, light, fight, hello, apple  
(머신러닝은 글자가 아닌 숫자를 집어넣는 인코딩 작업 필요)
- 들어간 문자 g, r, e, a, t, f, k, i, h, l, o, p

## 데이터 만들기
- great, right, light, freak, hello, apple, fightm
- [[h,e,l,l,o], [r,i,g,h,t],[l,i,g,h,t],[f,r,e,a,k],[f,i,g,h,t],[a,p,p,l,e], [g,r,e,a,t]]

In [4]:
import numpy as np
data = np.array([
    [g,r,e,a,t],
    [r,i,g,h,t],
    [l,i,g,h,t],
    [f,i,g,h,t],
    [f,r,e,a,k],
    [a,p,p,l,e],
    [h,e,l,l,o]
])

#### 원-핫 인코딩

In [5]:
# 12개의 알파벳 원핫인코딩
g = [1,0,0,0,0,0,0,0,0,0,0,0]
r = [0,1,0,0,0,0,0,0,0,0,0,0]
e = [0,0,1,0,0,0,0,0,0,0,0,0]
a = [0,0,0,1,0,0,0,0,0,0,0,0]
t = [0,0,0,0,1,0,0,0,0,0,0,0]
i = [0,0,0,0,0,1,0,0,0,0,0,0]
h = [0,0,0,0,0,0,1,0,0,0,0,0]
l = [0,0,0,0,0,0,0,1,0,0,0,0]
f = [0,0,0,0,0,0,0,0,1,0,0,0]
k = [0,0,0,0,0,0,0,0,0,1,0,0]
o = [0,0,0,0,0,0,0,0,0,0,1,0]
p = [0,0,0,0,0,0,0,0,0,0,0,1]

In [6]:
data.shape
# 데이터 갯수, 한 데이터 안의 변수의 개수, 변수의 개수
# sample, timestamp, feature

(7, 5, 12)

In [7]:
X_train = data[:,:-1]
y_train = data[:,-1]

In [8]:
X_train.shape, y_train.shape

((7, 4, 12), (7, 12))

## 모델 설계

In [10]:
# 모델링
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN

model = Sequential()

model.add(SimpleRNN(units = 10, input_shape=(4,12)))   # input_shape=(timestamp, feature) / 기본 activation이 tanh(하이퍼볼릭탄젠트)

model.add(Dense(12, activation='softmax'))   # 원핫인코딩한걸 라벨 값으로 가지고 있으니까 softmax

model.summary()

Init Plugin
Init Graph Optimizer
Init Kernel
Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2023-02-14 19:52:46.800674: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-14 19:52:46.800807: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 10)                230       
_________________________________________________________________
dense (Dense)                (None, 12)                132       
Total params: 362
Trainable params: 362
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
h1 = model.fit(X_train, y_train, epochs=500)

2023-02-14 19:52:51.429606: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2023-02-14 19:52:51.432425: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/500


2023-02-14 19:52:51.643037: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 1s/step - loss: 2.5456 - accuracy: 0.0000e+00
Epoch 2/500
1/1 [==============================] - 0s 30ms/step - loss: 2.5315 - accuracy: 0.0000e+00
Epoch 3/500
1/1 [==============================] - 0s 25ms/step - loss: 2.5175 - accuracy: 0.0000e+00
Epoch 4/500
1/1 [==============================] - 0s 24ms/step - loss: 2.5036 - accuracy: 0.0000e+00
Epoch 5/500
1/1 [==============================] - 0s 25ms/step - loss: 2.4898 - accuracy: 0.0000e+00
Epoch 6/500
1/1 [==============================] - 0s 23ms/step - loss: 2.4761 - accuracy: 0.0000e+00
Epoch 7/500
1/1 [==============================] - 0s 22ms/step - loss: 2.4625 - accuracy: 0.0000e+00
Epoch 8/500
1/1 [==============================] - 0s 21ms/step - loss: 2.4489 - accuracy: 0.0000e+00
Epoch 9/500
1/1 [==============================] - 0s 23ms/step - loss: 2.4355 - accuracy: 0.0000e+00
Epoch 10/500
1/1 [==============================] - 0s 53ms/step - loss: 2.4221 - accuracy: 0.00

In [ ]:
# g,  r,  e,  a,  t,  i,  h,  l,  f,  k,  o,  p
# 1,  2,  3,  4,  5,  6,  7,  8,  9,  10, 11, 12

In [13]:
X_test = np.array([[h,e,l,l]])

In [14]:
pre = model.predict(X_test)
p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12 = pre[0][0],pre[0][1],pre[0][2],pre[0][3],pre[0][4],pre[0][5],pre[0][6],pre[0][7],pre[0][8],pre[0][9],pre[0][10],pre[0][11]
list1 = [p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11,p12]
if max(list1) == p1:
  print('g')
elif max(list1) == p2:
  print('r')
elif max(list1) == p3:
  print('e')
elif max(list1) == p4:
  print('a')
elif max(list1) == p5:
  print('t')
elif max(list1) == p6:
  print('i')
elif max(list1) == p7:
  print('h')
elif max(list1) == p8:
  print('l')
elif max(list1) == p9:
  print('f')
elif max(list1) == p10:
  print('k')
elif max(list1) == p11:
  print('o')
elif max(list1) == p12:
  print('p')

2023-02-14 19:53:19.134886: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


o
